# Agent Simple Movement Demo

This interactive notebook demonstrates basic agent movement in a Gaussian odor field environment. You'll learn how to:

- Create synthetic odor environments with Gaussian plume sources
- Initialize navigation agents with different movement parameters
- Generate real-time animations using matplotlib
- Configure experiments with Hydra for reproducible research
- Use the seed manager for consistent results across sessions

## Learning Objectives

By the end of this demo, you will understand:
1. **Odor Field Generation**: How to create realistic synthetic odor environments
2. **Agent Navigation**: Basic movement mechanics and parameter effects
3. **Visualization Systems**: Real-time animation techniques for navigation research
4. **Configuration Management**: Using Hydra Compose API for dynamic parameter assembly
5. **Reproducibility**: Ensuring consistent experimental results with seed management

## 1. Setup and Imports

First, let's import the necessary modules from the refactored project structure:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML
import time
from pathlib import Path

# Import from new project structure
from {{cookiecutter.project_slug}}.core.navigator import NavigatorProtocol
from {{cookiecutter.project_slug}}.utils.visualization import SimulationVisualization
from {{cookiecutter.project_slug}}.utils.seed_manager import set_global_seed, get_current_seed
from {{cookiecutter.project_slug}}.config.schemas import SingleAgentConfig

# Hydra for dynamic configuration
try:
    from hydra import compose, initialize_config_store
    from hydra.core.config_store import ConfigStore
    from omegaconf import DictConfig, OmegaConf
    HYDRA_AVAILABLE = True
    print("✓ Hydra available for dynamic configuration")
except ImportError:
    HYDRA_AVAILABLE = False
    print("⚠ Hydra not available, using basic configuration")

# Enable inline plotting
%matplotlib widget
plt.style.use('default')

print("✓ All imports successful!")
print(f"✓ NumPy version: {np.__version__}")
print(f"✓ Matplotlib backend: {plt.get_backend()}")

## 2. Initialize Reproducible Environment

Set up the seed manager to ensure reproducible results across notebook sessions:

In [ ]:
# Initialize seed manager for reproducible experiments
DEMO_SEED = 42
actual_seed = set_global_seed(
    seed=DEMO_SEED,
    config={
        "validate_initialization": True,
        "log_seed_context": True,
        "preserve_state": True,
        "hash_environment": True
    }
)

print(f"🎯 Global seed set to: {actual_seed}")
print(f"🔄 Reproducible environment initialized")

# Display seed information for documentation
display(HTML(f"""
<div style="background-color: #e7f3ff; padding: 10px; border-left: 4px solid #007acc; margin: 10px 0;">
    <strong>🔒 Reproducibility Info:</strong><br>
    Global seed: <code>{actual_seed}</code><br>
    All random operations in this notebook will produce consistent results.
</div>
"""))

## 3. Odor Environment Generation

Learn how to create realistic synthetic odor environments using Gaussian plume models. This is fundamental for navigation research as it provides controllable, reproducible environments for testing algorithms.

In [ ]:
def create_gaussian_odor_field(width: int = 50, height: int = 50, centers=None, normalize: bool = True):
    """
    Create a Gaussian odor field with multiple sources.
    
    This function generates a 2D odor concentration field using Gaussian distributions
    to simulate realistic odor plume dispersion. Each source is characterized by:
    - Position (x, y): Location of the odor source
    - Intensity: Maximum concentration at the source
    - Sigma: Spread/dispersion parameter (larger = wider plume)
    
    Args:
        width: Environment width in grid units
        height: Environment height in grid units
        centers: List of (x, y, intensity, sigma) tuples for each odor source
        normalize: Whether to normalize concentrations to [0, 1] range
    
    Returns:
        2D numpy array with odor concentration values
    """
    if centers is None:
        # Default configuration: two odor sources with different characteristics
        centers = [
            (width * 0.7, height * 0.6, 1.0, 5.0),   # Strong, narrow source
            (width * 0.3, height * 0.3, 0.7, 7.0),   # Moderate, wider source
        ]
    
    # Create coordinate grids
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    odor_field = np.zeros((height, width), dtype=np.float32)
    
    # Add each Gaussian odor source
    for cx, cy, intensity, sigma in centers:
        # Gaussian formula: I * exp(-((x-cx)² + (y-cy)²) / (2σ²))
        gaussian = intensity * np.exp(-((x-cx)**2 + (y-cy)**2) / (2*sigma**2))
        odor_field += gaussian
    
    # Normalize to [0, 1] range if requested
    if normalize:
        max_val = np.max(odor_field)
        if max_val > 0:
            odor_field /= max_val
    
    return odor_field

# Create example odor field
example_field = create_gaussian_odor_field()
print(f"✓ Created odor field with shape: {example_field.shape}")
print(f"✓ Concentration range: [{np.min(example_field):.3f}, {np.max(example_field):.3f}]")
print(f"✓ Mean concentration: {np.mean(example_field):.3f}")

### Visualize the Odor Environment

Let's examine the structure of our synthetic odor field:

In [ ]:
# Create a figure to visualize the odor field structure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 2D heatmap
im1 = ax1.imshow(example_field, origin='lower', cmap='viridis', extent=[0, 50, 0, 50])
ax1.set_title('Odor Concentration Field\n(Top View)', fontsize=12, fontweight='bold')
ax1.set_xlabel('X Position')
ax1.set_ylabel('Y Position')
cbar1 = plt.colorbar(im1, ax=ax1)
cbar1.set_label('Concentration', rotation=270, labelpad=15)

# 3D surface plot
from mpl_toolkits.mplot3d import Axes3D
ax2 = fig.add_subplot(122, projection='3d')
x_surf, y_surf = np.meshgrid(np.arange(50), np.arange(50))
surf = ax2.plot_surface(x_surf, y_surf, example_field, cmap='viridis', alpha=0.8)
ax2.set_title('3D Odor Landscape', fontsize=12, fontweight='bold')
ax2.set_xlabel('X Position')
ax2.set_ylabel('Y Position')
ax2.set_zlabel('Concentration')

plt.tight_layout()
plt.show()

print("📊 The odor field shows two Gaussian sources with different intensities and spreads")
print("📊 Agents will navigate in this environment, sampling odor concentrations at their positions")

## 4. Agent Configuration with Hydra

Configure navigation agents using Hydra's Compose API for dynamic parameter assembly:

In [ ]:
# Simple Navigator class that implements NavigatorProtocol
class SimpleNavigator:
    """Simple navigator implementation for educational demonstrations."""
    
    def __init__(self, position=(5, 5), orientation=45, speed=0.5, max_speed=1.0):
        self._position = np.array(position, dtype=float)
        self._orientation = float(orientation)
        self._speed = float(speed)
        self._max_speed = float(max_speed)
        self._angular_velocity = 0.0
        
        # Store initial values for reset
        self._initial_position = self._position.copy()
        self._initial_orientation = self._orientation
        self._initial_speed = self._speed
    
    @property
    def positions(self) -> np.ndarray:
        """Get current position as shape (1, 2) array."""
        return self._position.reshape(1, -1)
    
    @property
    def orientations(self) -> np.ndarray:
        """Get current orientation as shape (1,) array."""
        return np.array([self._orientation])
    
    @property
    def speeds(self) -> np.ndarray:
        """Get current speed as shape (1,) array."""
        return np.array([self._speed])
    
    @property
    def max_speeds(self) -> np.ndarray:
        """Get max speed as shape (1,) array."""
        return np.array([self._max_speed])
    
    @property
    def angular_velocities(self) -> np.ndarray:
        """Get angular velocity as shape (1,) array."""
        return np.array([self._angular_velocity])
    
    @property
    def num_agents(self) -> int:
        """Number of agents (always 1 for SimpleNavigator)."""
        return 1
    
    def get_position(self) -> tuple:
        """Get current position as tuple for backward compatibility."""
        return tuple(self._position)
    
    @property
    def orientation(self) -> float:
        """Get current orientation for backward compatibility."""
        return self._orientation
    
    @property
    def speed(self) -> float:
        """Get current speed for backward compatibility."""
        return self._speed
    
    def set_orientation(self, orientation: float):
        """Set orientation angle in degrees."""
        self._orientation = orientation % 360.0
    
    def set_speed(self, speed: float):
        """Set speed, clamping to max_speed."""
        self._speed = max(0.0, min(speed, self._max_speed))
    
    def update(self, dt: float):
        """Update position based on current speed and orientation."""
        # Convert orientation to radians
        angle_rad = np.radians(self._orientation)
        
        # Calculate displacement
        dx = self._speed * np.cos(angle_rad) * dt
        dy = self._speed * np.sin(angle_rad) * dt
        
        # Update position
        self._position += np.array([dx, dy])
    
    def reset(self, **kwargs):
        """Reset to initial state."""
        self._position = self._initial_position.copy()
        self._orientation = self._initial_orientation
        self._speed = self._initial_speed
        self._angular_velocity = 0.0
    
    def step(self, env_array: np.ndarray):
        """Protocol-required step method."""
        self.update(0.5)  # Use fixed timestep
    
    def sample_odor(self, env_array: np.ndarray) -> float:
        """Sample odor at current position."""
        return self.read_single_antenna_odor(env_array)
    
    def read_single_antenna_odor(self, env_array: np.ndarray) -> float:
        """Read odor concentration at current position using bilinear interpolation."""
        x, y = self._position
        height, width = env_array.shape
        
        # Clamp to environment bounds
        x = max(0, min(x, width - 1))
        y = max(0, min(y, height - 1))
        
        # Bilinear interpolation
        x0, y0 = int(x), int(y)
        x1, y1 = min(x0 + 1, width - 1), min(y0 + 1, height - 1)
        
        if x0 == x1 and y0 == y1:
            return float(env_array[y0, x0])
        
        # Interpolation weights
        wx = x - x0
        wy = y - y0
        
        # Bilinear interpolation
        val = (env_array[y0, x0] * (1 - wx) * (1 - wy) +
               env_array[y0, x1] * wx * (1 - wy) +
               env_array[y1, x0] * (1 - wx) * wy +
               env_array[y1, x1] * wx * wy)
        
        return float(val)
    
    def sample_multiple_sensors(self, env_array, sensor_distance=5.0, sensor_angle=45.0, num_sensors=2, layout_name=None):
        """Sample multiple sensors around the agent."""
        # Simple implementation returning array of current position reading
        odor_val = self.read_single_antenna_odor(env_array)
        return np.array([odor_val] * num_sensors)

print("✓ SimpleNavigator class defined with NavigatorProtocol interface")

### Hydra Configuration Assembly

Use Hydra's Compose API to dynamically create and modify agent configurations:

In [ ]:
if HYDRA_AVAILABLE:
    # Register configuration schemas with Hydra ConfigStore
    cs = ConfigStore.instance()
    cs.store(name="agent_config", node=SingleAgentConfig)
    
    # Create base configuration using Hydra Compose API
    base_config = OmegaConf.create({
        "agent": {
            "position": [5.0, 5.0],
            "orientation": 45.0,
            "speed": 0.5,
            "max_speed": 1.0,
            "angular_velocity": 0.0
        },
        "environment": {
            "width": 50,
            "height": 50,
            "odor_sources": [
                {"x": 35.0, "y": 30.0, "intensity": 1.0, "sigma": 5.0},
                {"x": 15.0, "y": 15.0, "intensity": 0.7, "sigma": 7.0}
            ]
        },
        "simulation": {
            "num_frames": 40,
            "dt": 0.5,
            "animation_interval": 200
        }
    })
    
    print("✓ Hydra configuration initialized")
    print(f"✓ Agent starting position: {base_config.agent.position}")
    print(f"✓ Agent orientation: {base_config.agent.orientation}°")
    print(f"✓ Environment size: {base_config.environment.width}×{base_config.environment.height}")
    
else:
    # Fallback configuration without Hydra
    base_config = {
        "agent": {
            "position": [5.0, 5.0],
            "orientation": 45.0,
            "speed": 0.5,
            "max_speed": 1.0,
            "angular_velocity": 0.0
        },
        "environment": {
            "width": 50,
            "height": 50,
            "odor_sources": [
                {"x": 35.0, "y": 30.0, "intensity": 1.0, "sigma": 5.0},
                {"x": 15.0, "y": 15.0, "intensity": 0.7, "sigma": 7.0}
            ]
        },
        "simulation": {
            "num_frames": 40,
            "dt": 0.5,
            "animation_interval": 200
        }
    }
    print("✓ Basic configuration initialized (Hydra fallback)")

## 5. Interactive Parameter Exploration

Use interactive widgets to explore how different agent parameters affect movement behavior:

In [ ]:
# Create interactive widgets for parameter exploration
def create_parameter_widgets():
    """Create interactive widgets for agent parameter exploration."""
    
    # Agent parameter widgets
    start_x_widget = widgets.FloatSlider(
        value=5.0, min=0.0, max=50.0, step=1.0,
        description='Start X:',
        style={'description_width': 'initial'}
    )
    
    start_y_widget = widgets.FloatSlider(
        value=5.0, min=0.0, max=50.0, step=1.0,
        description='Start Y:',
        style={'description_width': 'initial'}
    )
    
    orientation_widget = widgets.FloatSlider(
        value=45.0, min=0.0, max=360.0, step=15.0,
        description='Orientation (°):',
        style={'description_width': 'initial'}
    )
    
    speed_widget = widgets.FloatSlider(
        value=0.5, min=0.0, max=2.0, step=0.1,
        description='Speed:',
        style={'description_width': 'initial'}
    )
    
    max_speed_widget = widgets.FloatSlider(
        value=1.0, min=0.1, max=3.0, step=0.1,
        description='Max Speed:',
        style={'description_width': 'initial'}
    )
    
    # Environment parameter widgets
    env_width_widget = widgets.IntSlider(
        value=50, min=20, max=100, step=10,
        description='Env Width:',
        style={'description_width': 'initial'}
    )
    
    env_height_widget = widgets.IntSlider(
        value=50, min=20, max=100, step=10,
        description='Env Height:',
        style={'description_width': 'initial'}
    )
    
    # Simulation parameter widgets
    num_frames_widget = widgets.IntSlider(
        value=40, min=10, max=100, step=10,
        description='Num Frames:',
        style={'description_width': 'initial'}
    )
    
    dt_widget = widgets.FloatSlider(
        value=0.5, min=0.1, max=2.0, step=0.1,
        description='Time Step:',
        style={'description_width': 'initial'}
    )
    
    # Group widgets in tabs
    agent_tab = widgets.VBox([
        widgets.HTML("<h4>🤖 Agent Parameters</h4>"),
        start_x_widget, start_y_widget, orientation_widget, speed_widget, max_speed_widget
    ])
    
    env_tab = widgets.VBox([
        widgets.HTML("<h4>🌍 Environment Parameters</h4>"),
        env_width_widget, env_height_widget
    ])
    
    sim_tab = widgets.VBox([
        widgets.HTML("<h4>⚙️ Simulation Parameters</h4>"),
        num_frames_widget, dt_widget
    ])
    
    tab_widget = widgets.Tab(children=[agent_tab, env_tab, sim_tab])
    tab_widget.set_title(0, 'Agent')
    tab_widget.set_title(1, 'Environment')
    tab_widget.set_title(2, 'Simulation')
    
    return {
        'tab_widget': tab_widget,
        'start_x': start_x_widget,
        'start_y': start_y_widget,
        'orientation': orientation_widget,
        'speed': speed_widget,
        'max_speed': max_speed_widget,
        'env_width': env_width_widget,
        'env_height': env_height_widget,
        'num_frames': num_frames_widget,
        'dt': dt_widget
    }

# Create widgets
widgets_dict = create_parameter_widgets()
display(widgets_dict['tab_widget'])

print("🎛️ Interactive parameter widgets created!")
print("🎛️ Adjust parameters above and run the cells below to see the effects")

## 6. Real-Time Animation Demo

Now let's create a real-time animation showing agent movement in the odor field. This demonstrates the core functionality of the navigation system:

In [ ]:
def run_movement_demo(widgets_dict):
    """
    Run the interactive movement demonstration.
    
    This function demonstrates how to:
    1. Create a synthetic odor environment
    2. Initialize a navigator with custom parameters
    3. Generate real-time animations
    4. Track agent state over time
    """
    
    # Get current widget values
    start_pos = (widgets_dict['start_x'].value, widgets_dict['start_y'].value)
    orientation = widgets_dict['orientation'].value
    speed = widgets_dict['speed'].value
    max_speed = widgets_dict['max_speed'].value
    env_width = widgets_dict['env_width'].value
    env_height = widgets_dict['env_height'].value
    num_frames = widgets_dict['num_frames'].value
    dt = widgets_dict['dt'].value
    
    print(f"🚀 Starting demo with agent at {start_pos}, orientation {orientation}°")
    
    # Create odor environment based on current parameters
    odor_centers = [
        (env_width * 0.7, env_height * 0.6, 1.0, 5.0),
        (env_width * 0.3, env_height * 0.3, 0.7, 7.0),
    ]
    odor_field = create_gaussian_odor_field(
        width=env_width, height=env_height, centers=odor_centers
    )
    
    # Create navigator with current parameters
    navigator = SimpleNavigator(
        position=start_pos,
        orientation=orientation,
        speed=speed,
        max_speed=max_speed
    )
    
    # Create visualization with performance optimization
    viz_config = {
        "animation": {
            "fps": 15,
            "interval": 67,  # ~15 FPS
            "blit": True,
            "figsize": [10, 8]
        },
        "theme": {
            "colormap": "viridis",
            "grid": True
        },
        "agents": {
            "marker_size": 100,
            "trail_length": 20,
            "orientation_arrow_scale": 2.0
        }
    }
    
    viz = SimulationVisualization(config=viz_config)
    viz.setup_environment(odor_field)
    
    # Storage for trajectory data
    trajectory_data = {
        'positions': [],
        'orientations': [],
        'odor_values': [],
        'speeds': [],
        'timestamps': []
    }
    
    start_time = time.time()
    
    # Define the update function for each animation frame
    def update_frame(frame_num):
        """Update function called for each animation frame."""
        current_time = time.time() - start_time
        
        # Update agent position
        navigator.update(dt)
        
        # Sample odor at current position
        current_pos = navigator.get_position()
        odor_value = navigator.read_single_antenna_odor(odor_field)
        
        # Store trajectory data
        trajectory_data['positions'].append(current_pos)
        trajectory_data['orientations'].append(navigator.orientation)
        trajectory_data['odor_values'].append(odor_value)
        trajectory_data['speeds'].append(navigator.speed)
        trajectory_data['timestamps'].append(current_time)
        
        # Print progress information
        if frame_num % 10 == 0 or frame_num < 5:
            print(f"Frame {frame_num:2d}: Pos=({current_pos[0]:.1f}, {current_pos[1]:.1f}), "
                  f"Orient={navigator.orientation:.0f}°, Odor={odor_value:.3f}")
        
        # Return data for visualization: (positions, orientations, odor_values)
        return (np.array([current_pos]), np.array([navigator.orientation]), np.array([odor_value]))
    
    # Create and run the animation
    print(f"🎬 Creating animation with {num_frames} frames...")
    anim = viz.create_animation(
        update_func=update_frame,
        frames=num_frames,
        interval=67  # ~15 FPS for smooth notebook performance
    )
    
    # Display animation
    viz.show()
    
    # Return trajectory data for analysis
    return trajectory_data, anim

# Button to run the demo
run_button = widgets.Button(
    description='🚀 Run Movement Demo',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

output_area = widgets.Output()

def on_run_button_clicked(b):
    """Handle run button click."""
    with output_area:
        output_area.clear_output()
        
        # Ensure reproducible results
        set_global_seed(DEMO_SEED)
        
        try:
            trajectory_data, anim = run_movement_demo(widgets_dict)
            
            # Store results for later analysis
            globals()['last_trajectory'] = trajectory_data
            globals()['last_animation'] = anim
            
            print("\n✅ Demo completed successfully!")
            print(f"📊 Trajectory data stored in 'last_trajectory' variable")
            print(f"🎬 Animation stored in 'last_animation' variable")
            
        except Exception as e:
            print(f"❌ Error running demo: {e}")
            import traceback
            traceback.print_exc()

run_button.on_click(on_run_button_clicked)

display(widgets.VBox([
    widgets.HTML("<h4>🎬 Run Animation Demo</h4>"),
    widgets.HTML("<p>Click the button below to run the movement demo with current parameters:</p>"),
    run_button,
    output_area
]))

print("🎮 Interactive demo ready! Adjust parameters above and click 'Run Movement Demo'")

## 7. Trajectory Analysis and Visualization

After running the demo, analyze the agent's trajectory and behavior:

In [ ]:
def analyze_trajectory():
    """Analyze the trajectory data from the last demo run."""
    
    if 'last_trajectory' not in globals():
        print("❌ No trajectory data available. Please run the movement demo first.")
        return
    
    trajectory = globals()['last_trajectory']
    
    # Convert to numpy arrays for analysis
    positions = np.array(trajectory['positions'])
    orientations = np.array(trajectory['orientations'])
    odor_values = np.array(trajectory['odor_values'])
    speeds = np.array(trajectory['speeds'])
    timestamps = np.array(trajectory['timestamps'])
    
    print(f"📊 Trajectory Analysis Summary")
    print(f"═" * 40)
    print(f"Total steps: {len(positions)}")
    print(f"Duration: {timestamps[-1]:.2f} seconds")
    print(f"Start position: ({positions[0][0]:.1f}, {positions[0][1]:.1f})")
    print(f"End position: ({positions[-1][0]:.1f}, {positions[-1][1]:.1f})")
    
    # Calculate trajectory metrics
    distances = np.linalg.norm(np.diff(positions, axis=0), axis=1)
    total_distance = np.sum(distances)
    straight_line_distance = np.linalg.norm(positions[-1] - positions[0])
    
    print(f"\n📏 Movement Metrics")
    print(f"─" * 30)
    print(f"Total distance traveled: {total_distance:.2f} units")
    print(f"Straight-line distance: {straight_line_distance:.2f} units")
    print(f"Path efficiency: {straight_line_distance/total_distance*100:.1f}%")
    print(f"Average speed: {np.mean(speeds):.3f} units/step")
    
    print(f"\n🧭 Orientation Analysis")
    print(f"─" * 30)
    orientation_changes = np.abs(np.diff(orientations))
    # Handle wraparound (e.g., 359° to 1°)
    orientation_changes = np.minimum(orientation_changes, 360 - orientation_changes)
    total_rotation = np.sum(orientation_changes)
    print(f"Initial orientation: {orientations[0]:.1f}°")
    print(f"Final orientation: {orientations[-1]:.1f}°")
    print(f"Total rotation: {total_rotation:.1f}°")
    print(f"Average orientation change: {np.mean(orientation_changes):.2f}°/step")
    
    print(f"\n🌬️ Odor Sampling Analysis")
    print(f"─" * 30)
    print(f"Initial odor concentration: {odor_values[0]:.4f}")
    print(f"Final odor concentration: {odor_values[-1]:.4f}")
    print(f"Maximum odor encountered: {np.max(odor_values):.4f}")
    print(f"Average odor concentration: {np.mean(odor_values):.4f}")
    print(f"Odor gradient (final - initial): {odor_values[-1] - odor_values[0]:.4f}")
    
    # Create detailed trajectory plots
    create_trajectory_plots(positions, orientations, odor_values, timestamps)

def create_trajectory_plots(positions, orientations, odor_values, timestamps):
    """Create detailed trajectory analysis plots."""
    
    fig = plt.figure(figsize=(15, 10))
    
    # 1. 2D trajectory with color-coded odor values
    ax1 = plt.subplot(2, 3, 1)
    scatter = ax1.scatter(positions[:, 0], positions[:, 1], c=odor_values, 
                         cmap='viridis', s=30, alpha=0.8)
    ax1.plot(positions[:, 0], positions[:, 1], 'k-', alpha=0.3, linewidth=1)
    ax1.scatter(positions[0, 0], positions[0, 1], c='red', s=100, marker='o', 
               label='Start', edgecolors='black', linewidth=2)
    ax1.scatter(positions[-1, 0], positions[-1, 1], c='blue', s=100, marker='s', 
               label='End', edgecolors='black', linewidth=2)
    plt.colorbar(scatter, ax=ax1, label='Odor Concentration')
    ax1.set_title('2D Trajectory\n(Color = Odor Concentration)', fontweight='bold')
    ax1.set_xlabel('X Position')
    ax1.set_ylabel('Y Position')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_aspect('equal')
    
    # 2. Odor concentration over time
    ax2 = plt.subplot(2, 3, 2)
    ax2.plot(timestamps, odor_values, 'b-', linewidth=2, marker='o', markersize=4)
    ax2.set_title('Odor Concentration vs Time', fontweight='bold')
    ax2.set_xlabel('Time (seconds)')
    ax2.set_ylabel('Odor Concentration')
    ax2.grid(True, alpha=0.3)
    ax2.fill_between(timestamps, odor_values, alpha=0.3)
    
    # 3. Orientation over time
    ax3 = plt.subplot(2, 3, 3)
    ax3.plot(timestamps, orientations, 'r-', linewidth=2, marker='s', markersize=4)
    ax3.set_title('Orientation vs Time', fontweight='bold')
    ax3.set_xlabel('Time (seconds)')
    ax3.set_ylabel('Orientation (degrees)')
    ax3.grid(True, alpha=0.3)
    ax3.set_ylim(0, 360)
    
    # 4. Speed analysis
    ax4 = plt.subplot(2, 3, 4)
    distances = np.linalg.norm(np.diff(positions, axis=0), axis=1)
    speed_from_pos = distances / np.diff(timestamps)
    ax4.plot(timestamps[1:], speed_from_pos, 'g-', linewidth=2, label='From Position')
    ax4.set_title('Speed Analysis', fontweight='bold')
    ax4.set_xlabel('Time (seconds)')
    ax4.set_ylabel('Speed (units/second)')
    ax4.grid(True, alpha=0.3)
    ax4.legend()
    
    # 5. Odor vs distance from start
    ax5 = plt.subplot(2, 3, 5)
    distances_from_start = np.linalg.norm(positions - positions[0], axis=1)
    ax5.scatter(distances_from_start, odor_values, c=timestamps, cmap='plasma', s=40, alpha=0.8)
    ax5.set_title('Odor vs Distance from Start\n(Color = Time)', fontweight='bold')
    ax5.set_xlabel('Distance from Start')
    ax5.set_ylabel('Odor Concentration')
    ax5.grid(True, alpha=0.3)
    
    # 6. Position components over time
    ax6 = plt.subplot(2, 3, 6)
    ax6.plot(timestamps, positions[:, 0], 'r-', linewidth=2, label='X Position', marker='o', markersize=3)
    ax6.plot(timestamps, positions[:, 1], 'b-', linewidth=2, label='Y Position', marker='s', markersize=3)
    ax6.set_title('Position Components vs Time', fontweight='bold')
    ax6.set_xlabel('Time (seconds)')
    ax6.set_ylabel('Position')
    ax6.grid(True, alpha=0.3)
    ax6.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("📈 Detailed trajectory plots generated!")

# Button to analyze trajectory
analyze_button = widgets.Button(
    description='📊 Analyze Trajectory',
    button_style='info',
    layout=widgets.Layout(width='200px', height='40px')
)

analysis_output = widgets.Output()

def on_analyze_button_clicked(b):
    """Handle analyze button click."""
    with analysis_output:
        analysis_output.clear_output()
        analyze_trajectory()

analyze_button.on_click(on_analyze_button_clicked)

display(widgets.VBox([
    widgets.HTML("<h4>📊 Trajectory Analysis</h4>"),
    widgets.HTML("<p>Analyze the agent's movement patterns and behavior:</p>"),
    analyze_button,
    analysis_output
]))

## 8. Configuration Export and Reproducibility

Learn how to export configurations for reproducible research:

In [ ]:
def export_experiment_config():
    """Export the current experiment configuration for reproducibility."""
    
    if 'last_trajectory' not in globals():
        print("❌ No experiment data available. Please run the movement demo first.")
        return
    
    # Get current widget values
    current_config = {
        "experiment": {
            "name": "simple_movement_demo",
            "version": "1.0",
            "timestamp": time.strftime("%Y-%m-%d_%H-%M-%S"),
            "seed": get_current_seed()
        },
        "agent": {
            "position": [widgets_dict['start_x'].value, widgets_dict['start_y'].value],
            "orientation": widgets_dict['orientation'].value,
            "speed": widgets_dict['speed'].value,
            "max_speed": widgets_dict['max_speed'].value,
            "angular_velocity": 0.0
        },
        "environment": {
            "width": widgets_dict['env_width'].value,
            "height": widgets_dict['env_height'].value,
            "odor_sources": [
                {
                    "x": widgets_dict['env_width'].value * 0.7,
                    "y": widgets_dict['env_height'].value * 0.6,
                    "intensity": 1.0,
                    "sigma": 5.0
                },
                {
                    "x": widgets_dict['env_width'].value * 0.3,
                    "y": widgets_dict['env_height'].value * 0.3,
                    "intensity": 0.7,
                    "sigma": 7.0
                }
            ]
        },
        "simulation": {
            "num_frames": widgets_dict['num_frames'].value,
            "dt": widgets_dict['dt'].value,
            "animation_interval": 200
        },
        "results": {
            "total_steps": len(globals()['last_trajectory']['positions']),
            "final_position": globals()['last_trajectory']['positions'][-1],
            "final_odor_value": globals()['last_trajectory']['odor_values'][-1]
        }
    }
    
    # Store configuration globally for reuse
    globals()['experiment_config'] = current_config
    
    print("✅ Experiment configuration exported!")
    print(f"🔄 Seed: {current_config['experiment']['seed']}")
    print(f"📅 Timestamp: {current_config['experiment']['timestamp']}")
    
    # Display configuration as formatted JSON
    import json
    config_json = json.dumps(current_config, indent=2)
    
    display(HTML(f"""
    <div style="background-color: #f8f9fa; padding: 15px; border: 1px solid #dee2e6; border-radius: 5px; margin: 10px 0;">
        <h5>📋 Experiment Configuration</h5>
        <pre style="background-color: #ffffff; padding: 10px; border: 1px solid #ced4da; border-radius: 3px; overflow-x: auto; font-size: 12px;">{config_json}</pre>
    </div>
    """))
    
    return current_config

def reproduce_experiment():
    """Reproduce the experiment using the exported configuration."""
    
    if 'experiment_config' not in globals():
        print("❌ No exported configuration available. Please export configuration first.")
        return
    
    config = globals()['experiment_config']
    
    print(f"🔄 Reproducing experiment with seed {config['experiment']['seed']}...")
    
    # Set seed for reproducibility
    set_global_seed(config['experiment']['seed'])
    
    # Apply configuration to widgets
    widgets_dict['start_x'].value = config['agent']['position'][0]
    widgets_dict['start_y'].value = config['agent']['position'][1]
    widgets_dict['orientation'].value = config['agent']['orientation']
    widgets_dict['speed'].value = config['agent']['speed']
    widgets_dict['max_speed'].value = config['agent']['max_speed']
    widgets_dict['env_width'].value = config['environment']['width']
    widgets_dict['env_height'].value = config['environment']['height']
    widgets_dict['num_frames'].value = config['simulation']['num_frames']
    widgets_dict['dt'].value = config['simulation']['dt']
    
    print("✅ Configuration applied to widgets")
    print("▶️ You can now run the movement demo to reproduce the exact same results")

# Create export/reproduce interface
export_button = widgets.Button(
    description='📋 Export Config',
    button_style='warning',
    layout=widgets.Layout(width='150px', height='35px')
)

reproduce_button = widgets.Button(
    description='🔄 Reproduce',
    button_style='primary',
    layout=widgets.Layout(width='150px', height='35px')
)

config_output = widgets.Output()

def on_export_button_clicked(b):
    with config_output:
        config_output.clear_output()
        export_experiment_config()

def on_reproduce_button_clicked(b):
    with config_output:
        config_output.clear_output()
        reproduce_experiment()

export_button.on_click(on_export_button_clicked)
reproduce_button.on_click(on_reproduce_button_clicked)

display(widgets.VBox([
    widgets.HTML("<h4>🔒 Reproducibility Tools</h4>"),
    widgets.HTML("<p>Export configurations and reproduce experiments:</p>"),
    widgets.HBox([export_button, reproduce_button]),
    config_output
]))

print("🔒 Reproducibility tools ready!")
print("📋 Export your current configuration after running experiments")
print("🔄 Reproduce exact results by applying saved configurations")

## 9. Summary and Next Steps

### What You've Learned

In this interactive demo, you've explored:

1. **🌍 Odor Environment Generation**: Created synthetic odor fields using Gaussian distributions
2. **🤖 Agent Navigation**: Configured agents with different movement parameters
3. **🎬 Real-time Visualization**: Generated animations showing agent movement
4. **⚙️ Dynamic Configuration**: Used Hydra Compose API for parameter management
5. **🔒 Reproducibility**: Ensured consistent results with seed management
6. **📊 Trajectory Analysis**: Analyzed movement patterns and behavior
7. **📋 Configuration Export**: Saved and reproduced experiment configurations

### Key Concepts

- **NavigatorProtocol**: Standardized interface for navigation implementations
- **SimulationVisualization**: High-performance animation system
- **Seed Management**: Reproducible random number generation
- **Hydra Configuration**: Dynamic parameter assembly and management
- **Bilinear Interpolation**: Accurate odor sampling at agent positions

### Next Steps

1. **Explore Advanced Demos**: Try other notebooks in the `demos/` folder
2. **Implement Custom Algorithms**: Create your own navigation strategies
3. **Multi-Agent Scenarios**: Experiment with multiple agents
4. **Real Video Data**: Use actual video plume data instead of synthetic fields
5. **Machine Learning Integration**: Apply RL or neural network approaches

### Troubleshooting

If you encounter issues:
- Check that all dependencies are properly installed
- Ensure matplotlib widget backend is enabled (`%matplotlib widget`)
- Verify seed manager initialization completed successfully
- Check console output for detailed error messages

### References

- [Project Documentation](../README.md)
- [Configuration Schemas](../../src/{{cookiecutter.project_slug}}/config/schemas.py)
- [Visualization Module](../../src/{{cookiecutter.project_slug}}/utils/visualization.py)
- [Hydra Configuration System](https://hydra.cc/)

Happy experimenting! 🚀